In [1]:
import numpy as np
import sklearn
import pandas as pd
import re
from keras.preprocessing import sequence
from keras.models import Sequential
import random
from numpy import loadtxt
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
import tldextract

ModuleNotFoundError: No module named 'keras'

In [2]:
data = pd.read_csv('data.csv')
data.head()

domain domain_type
0  000directory.com.ar      benign
1       000webhost.com      benign
2          001fans.com      benign
3   01-telecharger.com      benign
4       010shangpu.com      benign

In [3]:
#set benign to 0 and dga to 1
data['dt'] = pd.factorize(data['domain_type'])[0]
#data.dtypes 
#(do i need to set dt to catagory? find out later)

In [4]:
data['extract'] = data['domain'].apply(lambda url: tldextract.extract(url).domain)
X = data['extract']



In [5]:
# Create A Dicitonary of valid chars
unique_chars = enumerate(set(''.join(X))) 
chars_dict = dict()
for i, x in unique_chars: #index of enum starts with 0
    #print('i: ' + str(i) + '  x: ' + x)
    chars_dict[x] = i + 1 #leave 0 for padding

In [6]:
# print(list(enumerate(set(''.join(X))) ))
chars_dict

{'4': 1,
 'r': 2,
 'n': 3,
 'q': 4,
 'e': 5,
 'a': 6,
 '2': 7,
 'o': 8,
 'p': 9,
 'b': 10,
 'u': 11,
 'x': 12,
 '7': 13,
 'h': 14,
 '\n': 15,
 'c': 16,
 '9': 17,
 'w': 18,
 'i': 19,
 'j': 20,
 'l': 21,
 '5': 22,
 's': 23,
 'g': 24,
 'm': 25,
 't': 26,
 'd': 27,
 '3': 28,
 'v': 29,
 'k': 30,
 '-': 31,
 '0': 32,
 '6': 33,
 '8': 34,
 'z': 35,
 'y': 36,
 'f': 37,
 '1': 38}

In [7]:
#index 0 is also going to be a feature(padding/unknown).
max_features_num = len(chars_dict) + 1

#max length will be the max length of domain in our dataset
maxlen = np.max([len(x) for x in X])

In [8]:
maxlen

56

In [9]:
# Convert characters to int
X_in_int = []
for domain in X:
    domain_in_int = []
    for c in domain:
        domain_in_int.append(chars_dict[c])
    X_in_int.append(domain_in_int)

In [10]:
X_in_int[1]


[32, 32, 32, 18, 5, 10, 14, 8, 23, 26]

In [11]:
#pad to max length using keras... do we need to use a certain max len?
X = X_in_int
X = sequence.pad_sequences(X, maxlen=maxlen)
X.shape

(144945, 56)

In [12]:
Y = data.iloc[:, 2]
Y.head()
# len(Y)

0    0
1    0
2    0
3    0
4    0
Name: dt, dtype: int64

In [13]:
## Train Model - XGBOOST
# documentation https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/


In [14]:
seed = 11
test_size = .33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

#len(X_train)+len(X_test)

In [15]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)


In [16]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [17]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.55%


In [18]:
## PART two 
#https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

In [19]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

In [20]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 1000  # The number of training iterations


#https://xgboost.readthedocs.io/en/latest/parameter.html

In [21]:
model = xgb.train(param, D_train, steps)

In [22]:


preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.9868876679241059
Recall = 0.9856730239602758
Accuracy = 0.9868079946479344


In [23]:
## test accuracy on assignment

In [24]:
test = pd.read_csv('assignment.csv')

In [44]:
### INSERT THE TLDEXTRACT HERE as well

test['extract'] = test['domain'].apply(lambda url: tldextract.extract(url).domain)
XT = test['extract']

XT_in_int = []
for domain in XT:
    domain_in_int_t = []
    for c in domain:
        domain_in_int_t.append(chars_dict[c])
    XT_in_int.append(domain_in_int_t)

#need to stick the url split library in here
    # type(x1)
# x1.head()

In [45]:
XT_in_int[1]

[9, 9, 21, 8, 11, 21, 23, 8, 23, 8, 14, 29, 25, 30, 29]

In [46]:
XT = XT_in_int
XT = sequence.pad_sequences(XT, maxlen=maxlen)
XT.shape

(500, 56)

In [47]:
DT = xgb.DMatrix(XT)
preds_test = model.predict(DT)
preds_test

array([[6.35959566e-01, 3.64040434e-01],
       [6.72276318e-03, 9.93277252e-01],
       [6.48356149e-07, 9.99999404e-01],
       [2.36645997e-06, 9.99997616e-01],
       [9.99730766e-01, 2.69212091e-04],
       [9.85405028e-01, 1.45950140e-02],
       [8.73492041e-04, 9.99126494e-01],
       [9.99977112e-01, 2.28882927e-05],
       [3.58364218e-06, 9.99996424e-01],
       [4.53647226e-01, 5.46352744e-01],
       [9.26254392e-01, 7.37455860e-02],
       [7.03257648e-03, 9.92967486e-01],
       [9.60657895e-01, 3.93421352e-02],
       [2.62100138e-02, 9.73790050e-01],
       [9.99651670e-01, 3.48353264e-04],
       [1.25676580e-02, 9.87432361e-01],
       [9.99689460e-01, 3.10541800e-04],
       [4.16269928e-01, 5.83730102e-01],
       [3.56660187e-02, 9.64333951e-01],
       [1.16953022e-04, 9.99883056e-01],
       [9.71799910e-01, 2.82001123e-02],
       [3.27081739e-06, 9.99996781e-01],
       [9.97716784e-01, 2.28322553e-03],
       [9.98377800e-01, 1.62217370e-03],
       [5.786646

In [48]:
best_preds_test = np.asarray([np.argmax(line) for line in preds_test])
best_preds_test
# test['threat']=best_preds_test

# test.dtypes

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [49]:
# change 0 to benign and 1 to dga
con = []
for i in best_preds_test:
    if best_preds_test[i] == 0:
        con.append('benign')
    else:
        con.append('dga')

In [50]:
test['threat']=con
test

# test.to_csv (r'C:\Users\micha\Desktop\pyexpolore\test.csv', index = False, header=True)

domain  threat  \
0                    www.dutchwebdesign.com  benign   
1                   www.pploulsosohvmkv.com     dga   
2                    www.cfsjdqwdhmwkiv.com     dga   
3    www.uocoqgiusyeciouaimcauykqswsymo.com     dga   
4                     www.total-toolbar.com  benign   
..                                      ...     ...   
495                 www.u23lib5bcxw0yju.com     dga   
496                       www.prmysangh.com  benign   
497              www.ociqkeasqocgoecksu.com     dga   
498                            www.hj-9.com  benign   
499                          www.fpa-be.com  benign   

                            extract  
0                    dutchwebdesign  
1                   pploulsosohvmkv  
2                    cfsjdqwdhmwkiv  
3    uocoqgiusyeciouaimcauykqswsymo  
4                     total-toolbar  
..                              ...  
495                 u23lib5bcxw0yju  
496                       prmysangh  
497              ociqkeasqocgoecksu  
498                            hj-9  
499                          fpa-be  

[500 rows x 3 columns]